In [1]:
#!pip install -r requirements.txt
# Restart the kernel
import json, re
import pandas as pd
from tqdm import tqdm

## Load the Data

- Load the csv and json files in appropriate formats

In [2]:
df = pd.read_csv("oidv6-class-descriptions.csv")
df = df.set_index('LabelName')
class_dict = df.to_dict()['DisplayName']
with open("bbox_labels_600_hierarchy.json", "r") as f:
    data = json.load(f)

## Parse the JSON Data

- Parse the json data and convert Class code to Class names

In [3]:
def parse_json(json_object):
    '''Convert the class codes to class names   '''
    for jsn in json_object:
        if jsn == 'LabelName':
            json_object['LabelName'] = class_dict.get(json_object['LabelName']) if json_object['LabelName'] in class_dict else json_object['LabelName'] 
        elif jsn == 'Subcategory' or jsn == "Part":
            for j in json_object[jsn]:
                parse_json(j)
    return json_object       

data = parse_json(data)

## Create Hierarchical Tree

- Parse the JSON data and create a Tree structure
- We'll Use a custom Implementation of the Tree Datastructure and Anytree library's tree structure for the purpose

## Custom Implementation of the Tree Datastructure

- Use class in python to create a custom Tree Data structure

In [4]:
PARENT_DICT = dict() 
class Node:
        
    def __init__(self, name, parent=None):
        if parent is not None:
            parent.add_child(name)
        self.name = name
        self.parent = parent
        
    def display(self):
        route = self.name
        parent = self.parent
        while parent is not None:
            if parent.name != '/m/0bl9f':
                route = route + " -> " + parent.name
            parent = parent.parent
        return route
    
    def add_child(self, name):
        if self.name not in PARENT_DICT:
            PARENT_DICT[self.name] = {name}
        else:
            PARENT_DICT[self.name].add(name)
            
    @property     
    def children(self):
        return tuple(PARENT_DICT[self.name])

In [5]:
cv = lambda x: re.sub("[^a-zA-Z_]", '', x.lower().replace(' ', '_'))
def create_tree(json_object, parent):   
    '''Parse JSON Data and create Nodes with appropriate Parent'''
    for jsn in json_object:
        if jsn == 'LabelName':
            exec(f"{cv(json_object['LabelName'])} = Node(json_object['LabelName'], parent=parent)")
            parent.add_child(json_object['LabelName'])
        elif jsn == 'Subcategory' or jsn == "Part":
            for j in json_object[jsn]:
                exec(f"create_tree(j, {cv(json_object['LabelName'])})")  
    globals().update(locals())

TOP = Node("Open Image Classes")
create_tree(data, TOP)

In [6]:
def get_siblings(node):
    '''Returns names of all the nodes with same parent node as the given node.'''
    return node.parent.children

def get_parent(node):
    '''Returns the name of the parent node for a given node'''
    return node.parent.name

def get_ancestors(node):
    return tuple(x for x in node.display().split(' -> ') if x != node.name)

def same_ancestor(node1, node2):
    '''Returns True if two nodes share the same ancestory.'''
    return get_ancestors(node1) == get_ancestors(node2)

## Get Siblings

In [7]:
get_siblings(cowboy_hat)

('Cowboy hat', 'Fedora', 'Sombrero', 'Sun hat')

## Get Parent Node

In [8]:
get_parent(cowboy_hat)

'Hat'

## Get Ancestor Nodes 

In [9]:
get_ancestors(cowboy_hat)

('Hat', 'Fashion accessory', 'Clothing', 'Open Image Classes')

## Check if two nodes share the same Ancestory

In [10]:
same_ancestor(cowboy_hat, fedora)

True

In [11]:
same_ancestor(cowboy_hat, ceiling_fan)

False

## Use Anytree Python Library for Tree Data structure

- Use the Anytree python library for creating a Tree Data Structure

In [12]:
from anytree import Node, RenderTree

cv = lambda x: re.sub("[^a-zA-Z_]", '', x.lower().replace(' ', '_'))
def create_tree(json_object, parent):   
    '''Parse JSON Data and create Nodes with appropriate Parent'''
    for jsn in json_object:
        if jsn == 'LabelName':
            exec(f"{cv(json_object['LabelName'])} = Node(json_object['LabelName'], parent=parent)")
        elif jsn == 'Subcategory' or jsn == "Part":
            for j in json_object[jsn]:
                exec(f"create_tree(j, {cv(json_object['LabelName'])})")  
    globals().update(locals())

TOP = Node("Open Image Classes")
create_tree(data, TOP)

In [13]:
def get_siblings(node):
    '''Returns names of all the nodes with same parent node as the given node.'''
    return tuple(x.name for x in node.parent.children)

def get_parent(node):
    '''Returns the name of the parent node for a given node'''
    return node.parent.name

def get_ancestors(node):
    '''Returns names of all the ancestor node for a given node.'''
    # Filtering out the unwanted/unnamed node
    unwanted_ancestor =  '/m/0bl9f'
    ancestors = list()
    parent = node
    while node.parent is not None:
        parent = node.parent.name
        node = node.parent  
        if parent != unwanted_ancestor:
            ancestors.append(parent)
    return tuple(ancestors)

def same_ancestor(node1, node2):
    '''Returns True if two nodes share the same ancestory.'''
    return get_ancestors(node1) == get_ancestors(node2)

## Get Siblings

In [14]:
get_siblings(cowboy_hat)

('Cowboy hat', 'Fedora', 'Sombrero', 'Sun hat')

## Get Parent Node

In [15]:
get_parent(cowboy_hat)

'Hat'

## Get Ancestor Nodes 

In [16]:
get_ancestors(cowboy_hat)

('Hat', 'Fashion accessory', 'Clothing', 'Open Image Classes')

## Check if two nodes share the same Ancestory

In [17]:
same_ancestor(cowboy_hat, fedora)

True

In [18]:
same_ancestor(cowboy_hat, ceiling_fan)

False

## Visualize the Hierarchy for all the Classes

In [19]:
for pre, fill, node in RenderTree(TOP):
    print("%s%s" % (pre, node.name))

Open Image Classes
└── /m/0bl9f
    ├── Coin
    ├── Flag
    ├── Light bulb
    ├── Toy
    │   ├── Doll
    │   ├── Balloon
    │   ├── Dice
    │   ├── Flying disc
    │   ├── Kite
    │   └── Teddy bear
    ├── Home appliance
    │   ├── Washing machine
    │   ├── Toaster
    │   ├── Oven
    │   ├── Blender
    │   ├── Gas stove
    │   ├── Mechanical fan
    │   ├── Heater
    │   ├── Kettle
    │   ├── Hair dryer
    │   ├── Refrigerator
    │   ├── Wood-burning stove
    │   ├── Humidifier
    │   ├── Mixer
    │   ├── Coffeemaker
    │   ├── Microwave oven
    │   ├── Dishwasher
    │   ├── Sewing machine
    │   ├── Hand dryer
    │   └── Ceiling fan
    ├── Plumbing fixture
    │   ├── Sink
    │   ├── Bidet
    │   ├── Shower
    │   ├── Tap
    │   ├── Bathtub
    │   └── Toilet
    ├── Office supplies
    │   ├── Scissors
    │   ├── Poster
    │   ├── Calculator
    │   ├── Box
    │   ├── Stapler
    │   ├── Whiteboard
    │   ├── Pencil sharpener
    │   ├── Eraser
  